<a href="https://colab.research.google.com/github/wiemabbassi/Taquin-Solver/blob/main/Taquin_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!apt-get install -y xvfb
!pip install pygame PyVirtualDisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(300, 300))
display.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,813 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [ ]:
import pygame
import sys

def estEtatFinal(t):
    return t == [[1, 2, 3], [8, 0, 4], [7, 6, 5]]

def numero(t, x, y):
    return t[x][y]

def permuter(t, c1, c2):
    t[c1[0]][c1[1]], t[c2[0]][c2[1]] = t[c2[0]][c2[1]], t[c1[0]][c1[1]]

def position_case_vide(t):
    for i in range(3):
        for j in range(3):
            if t[i][j] == 0:
                return (i, j)

def afficher_taquin(t):
    for row in t:
        print(" ".join(map(str, row)))
    print("------------")

def transitions(etat):
    t = etat
    voisins = []
    x, y = position_case_vide(t)
    for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            nouveau_etat = [row[:] for row in t]
            permuter(nouveau_etat, (x, y), (nx, ny))
            voisins.append(nouveau_etat)
    return voisins

def DFSL(etat_depart, L):
    freeNodes = [(etat_depart, 0)]
    closedNodes = []
    goalNode = None
    explored_states = []
    nodes_explored = 0

    while freeNodes:
        currentNode, depth = freeNodes.pop(0)
        closedNodes.append(currentNode)
        if currentNode not in explored_states:
            explored_states.append(currentNode)

        if estEtatFinal(currentNode):
            goalNode = currentNode
            break

        if depth < L:
            generatedStates = transitions(currentNode)
            for s in generatedStates:
                nodes_explored += 1
                if s not in explored_states:
                    freeNodes.insert(0, (s, depth + 1))

    return goalNode, nodes_explored, len(closedNodes)

def draw_board(board):
    screen.fill(WHITE)
    for i in range(3):
        for j in range(3):
            pygame.draw.rect(screen, GRAY, [j * (CELL_SIZE + MARGIN), i * (CELL_SIZE + MARGIN), CELL_SIZE, CELL_SIZE])
            if board[i][j] != 0:
                text_surface = FONT.render(str(board[i][j]), True, BLACK)
                text_rect = text_surface.get_rect(center=(j * (CELL_SIZE + MARGIN) + CELL_SIZE // 2,
                                                          i * (CELL_SIZE + MARGIN) + CELL_SIZE // 2))
                screen.blit(text_surface, text_rect)

    pygame.display.flip()

def main():
    etat_initial = [[1, 2, 3], [8, 6, 0], [7, 5, 4]]
    print("État initial du taquin :")
    afficher_taquin(etat_initial)

    freeNodes = [(etat_initial, 0)]
    closedNodes = []
    goalNode = None
    explored_states = []
    nodes_explored = 0


    pygame.init()

    SCREEN_WIDTH = 300
    SCREEN_HEIGHT = 300
    global screen
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("8-Puzzle Solver")

    global CELL_SIZE, MARGIN, FONT, WHITE, BLACK, GRAY
    CELL_SIZE = 100
    MARGIN = 5
    FONT_SIZE = 36
    FONT = pygame.font.SysFont(None, FONT_SIZE)
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    GRAY = (200, 200, 200)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if freeNodes:
            current_node, depth = freeNodes.pop(0)
            closedNodes.append(current_node)
            if current_node not in explored_states:
                explored_states.append(current_node)

            if estEtatFinal(current_node):
                goalNode = current_node
                break

            if depth < 3:
                generatedStates = transitions(current_node)
                for s in generatedStates:
                    nodes_explored += 1
                    if s not in explored_states:
                        freeNodes.insert(0, (s, depth + 1))

            draw_board(current_node)
            pygame.time.delay(500)


    draw_board(goalNode)
    print("État final:")
    if goalNode is None:
        print("Pas de solutions")
    else:
        afficher_taquin(goalNode)
    print("Nombre d'états explorés:", nodes_explored)
    print("Nombre de nœuds clos:", len(closedNodes))

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

if __name__ == "__main__":
    main()


État initial du taquin :
1 2 3
8 6 0
7 5 4
------------
État final:
1 2 3
8 0 4
7 6 5
------------
Nombre d'états explorés: 26
Nombre de nœuds clos: 19
